# Battle of the neighborhoods
#### 1. A description of the problem and a discussion of the background. (15 marks)


The problem is to provide information to someone to help with a decision of where to move within the US. The scope will be the top 100 cities in the US by population. If the person who is moving has an idea of a city they like, for example, a city like Minneapolis, and wants to know other cities like that one, a cluster analysis could help identify the other similar cities within the same cluster as Minneapolis. Conversely, if the person moving knows they do NOT like Minneapolis, they could avoid moving to cities within the same cluster as it.

#### 2. A description of the data and how it will be used to solve the problem. (15 marks)

I will use data from wikipedia to provide list of the top 100 cities, including location (latitude and longitude), population, land area, population density, and growth rate. This data will have to be manipulated and cleaned up, extra characters removed, and converted to numeric data. I will also plot the city locations on a US map.

I will add to this data, additional data from Foursquare about the venues in each city. The venue categories will be the main point of comparison, specifically, the percent breakdown of each type of venue category. To do this, I'll have to convert the categories into dummies.

From there, I will normalize all the columns of the dataframe (population, area, growth, density, venue categories) for preparation into cluster analysis. I will run a few different cluster models and evaluate performance of these. Finally, I will provide the cluster results of similar cities, as well as an updated color-coded map.

https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population

In [1]:
import requests
import pandas as pd
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
try:
    import folium
except:
    !conda install -c conda-forge folium=0.5.0 --yes
    import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.1               |             py_0          26 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

#### Start by scraping Wikipedia page

In [2]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population', header=0)

In [81]:
df = tables[4]
# Limit to top 100 cities
df = df[:100]

#### Parse the latitude and longitude values, and add as new df columns. Drop the Location column.

In [85]:
latlon = df.Location.str.split('\ufeff / \ufeff')
latlon = [i[1].split(' ') for i in latlon]
lat = [float(i[0][:-2]) for i in latlon]
lon = [-float(i[1][:-2]) for i in latlon]

In [86]:
df['Latitude'] = lat
df['Longitude'] = lon
df = df.drop(['Location'])

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [87]:
df.head()

,2019rank,City,State[c],2019estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location,Latitude,Longitude
0,1,New York[d],New York,8336817,8175133,+1.98%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W,40.6635,-73.9387
1,2,Los Angeles,California,3979576,3792621,+4.93%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W,34.0194,-118.4108
2,3,Chicago,Illinois,2693976,2695598,−0.06%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W,41.8376,-87.6818
3,4,Houston[3],Texas,2320268,2100263,+10.48%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W,29.7866,-95.3909
4,5,Phoenix,Arizona,1680992,1445632,+16.28%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W,33.5722,-112.0901


In [38]:
na

'40°39′49″N 73°56′19″W'

#### Limit to top 100

#### Map of top 100 cities

In [ ]:
#Looked these up from Google
us_lat = 37.0902
us_long = -95.7129

# create map
map_us = folium.Map(location=[us_lat, us_long], zoom_start=5)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Postal Code']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_us)  
    
map_us

#### Foursquare parameters

In [ ]:
VERSION = '20200430' # Foursquare API version
LIMIT = 100

#### Foursquare credentials (hidden)

In [ ]:
# The code was removed by Watson Studio for sharing.

#### Create function to get nearby venues

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
us_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude'])
us_venues.head()

In [ ]:
us_venues.shape

#### Get all venue categories from Foursquare so groups can be combined

In [ ]:
url = 'https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET,
    VERSION)
categories = requests.get(url).json()['response']['categories']